### Librerias necesarias 
h3 geopandas folium matplotlib shapely pandas numpy

In [6]:
import geopandas as gpd
import h3
import folium
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

# Paso 1: Definir límites aproximados de Nueva York
# Coordenadas aproximadas que cubren el área de NYC (lat/lon)
nyc_bounds = {
    'min_lat': 40.5,
    'max_lat': 40.9,
    'min_lon': -74.05,
    'max_lon': -73.7
}

# Paso 2: Configurar resolución H3
# Resolución H3 (0-15, donde mayor número = hexágonos más pequeños)
# Resolución 8-9 suele ser buena para áreas urbanas
h3_resolution = 8

# Paso 3: Crear una lista de hexágonos H3 que cubren NYC
def generate_h3_hexagons(bounds, resolution):
    # Crear un polígono que represente los límites de NYC
    lats = [bounds['min_lat'], bounds['max_lat'], bounds['max_lat'], bounds['min_lat'], bounds['min_lat']]
    lons = [bounds['min_lon'], bounds['min_lon'], bounds['max_lon'], bounds['max_lon'], bounds['min_lon']]
    
    # Verificamos qué versión de h3 estamos usando
    # Para versiones más nuevas (h3-py v4+)
    if hasattr(h3, 'polyfill_geojson'):
        hexagons = h3.polyfill_geojson(
            {
                'type': 'Polygon',
                'coordinates': [[[lon, lat] for lon, lat in zip(lons, lats)]]
            }, 
            resolution
        )
    # Para versiones antiguas (h3-py v3.x)
    elif hasattr(h3, 'polyfill'):
        hexagons = h3.polyfill(
            {
                'type': 'Polygon',
                'coordinates': [[[lon, lat] for lon, lat in zip(lons, lats)]]
            }, 
            res=resolution
        )
    else:
        raise AttributeError("No se encontró método polyfill en el módulo h3. Verifica la versión instalada.")
    
    return list(hexagons)

# Paso 4: Convertir hexágonos a geometría para mapeo
def h3_to_polygons(h3_hexes):
    polygons = []
    for hex_id in h3_hexes:
        # Verificamos la versión de h3 para usar el método correcto
        if hasattr(h3, 'h3_to_geo_boundary'):
            # Versión antigua
            boundaries = h3.h3_to_geo_boundary(hex_id, geo_json=True)
        else:
            # Versión nueva (h3-py v4+)
            boundaries = h3.cell_to_boundary(hex_id, geo_json=True)
        
        # Convertir a formato de polígono Shapely
        polygons.append(Polygon(boundaries))
    
    return polygons

# Generar hexágonos
nyc_hexagons = generate_h3_hexagons(nyc_bounds, h3_resolution)
print(f"Número de hexágonos generados: {len(nyc_hexagons)}")

# Convertir a GeoDataFrame
hex_polygons = h3_to_polygons(nyc_hexagons)
gdf = gpd.GeoDataFrame(
    {'h3_id': nyc_hexagons, 'geometry': hex_polygons}, 
    crs="EPSG:4326"
)

# Paso 5: Crear un mapa interactivo con Folium
def create_folium_map(gdf):
    # Crear un mapa centrado en NYC
    m = folium.Map(
        location=[40.7128, -74.0060],  # Centro aproximado de NYC
        zoom_start=11,
        tiles="CartoDB positron"
    )
    
    # Añadir hexágonos al mapa
    folium.GeoJson(
        gdf,
        style_function=lambda x: {
            'fillColor': '#3186cc',
            'color': '#000000',
            'fillOpacity': 0.2,
            'weight': 0.5
        },
        tooltip=folium.GeoJsonTooltip(fields=['h3_id'], aliases=['H3 ID:'])
    ).add_to(m)
    
    return m

# Crear mapa interactivo
nyc_map = create_folium_map(gdf)

# Guardar mapa
nyc_map.save('nyc_h3_hexagons.html')

# Paso 6: Visualizar con matplotlib para vista estática
plt.figure(figsize=(12, 10))
gdf.plot(edgecolor='black', facecolor='none', alpha=0.5, linewidth=0.5)
plt.title(f'Mapa de Nueva York con hexágonos H3 (Resolución {h3_resolution})')
plt.axis('off')
plt.tight_layout()
plt.savefig('nyc_h3_hexagons.png', dpi=300)
plt.show()

AttributeError: No se encontró método polyfill en el módulo h3. Verifica la versión instalada.